In [12]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/arendjan/tmx-pico-aio.git@sensors
!{sys.executable} -m pip install nest_asyncio

  Cloning https://github.com/arendjan/tmx-pico-aio.git (to revision sensors) to c:\users\arendjanvanhil\appdata\local\temp\pip-req-build-xn3r_724
  Resolved https://github.com/arendjan/tmx-pico-aio.git to commit 98a214d262a9602b4ea6b33a635d58356411ad06
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
  Using cached aioserial-1.3.1-py3-none-any.whl
  Created wheel for tmx-pico-aio: filename=tmx_pico_aio-1.3-py3-none-any.whl size=32764 sha256=ae94e8a848d899096f4a52ee90e8abccd74f76b1e4230ffc8ad11979d2efe4d4
  Stored in directory: C:\Users\arendjanvanhil\AppData\Local\Temp\pip-ephem-wheel-cache-3g4lm2qc\wheels\1c\4f\5c\a235d1789f13b2c95cbcba16067f910096d5138

  Running command git clone --filter=blob:none --quiet https://github.com/arendjan/tmx-pico-aio.git 'C:\Users\arendjanvanhil\AppData\Local\Temp\pip-req-build-xn3r_724'
  Running command git checkout -b sensors --track origin/sensors
  branch 'sensors' set up to track 'origin/sensors'.
  Switched to a new branch 'sensors'


Create a board and connect your components:

In [13]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import struct
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

board = tmx_pico_aio.TmxPicoAio(loop=loop)

TelemetrixRpiPicoAio:  Version 1.5

Copyright (c) 2021 Alan Yorinks All Rights Reserved.

Opening all potential serial ports...
	COM13
Retrieving pico ID...
Pico Unique ID: [230, 97, 100, 8, 67, 88, 0, 0]
Telemetrix4RPiPico Version Number: 1.2


Add your callback, like printing or adding it to a list

In [17]:
MAX_ADC = 2**12
R1 = 10_000 # other resistor value, 10k
MAX_V = 3.3
ldr_vcc = True # is the ldr connected to VCC or the constant resistor

def calculate_ldr_resistance(adc_val):
    volt = adc_val/MAX_ADC*MAX_V
    # ratio = R1 : R_ldr
    ratio = volt/MAX_V if ldr_vcc else 1-volt/MAX_V
    factor = (1/ratio -1)
    ldr_res = factor * R1
    # print(f"a: {adc_val}  v: {volt} r: {ratio} fac: {factor}, ldr: {ldr_res}")
    return ldr_res


async def the_callback(data): # only will be called when there is a change
    adc_val = data[2]
    print(calculate_ldr_resistance(adc_val) )

a: 2048  v: 1.65 r: 0.5 fac: 1.0, ldr: 10000.0
a: 2730  v: 2.199462890625 r: 0.66650390625 fac: 0.5003663003663004, ldr: 5003.663003663004


Add the sensor to the system

In [15]:
async def addSensor(board):
    await asyncio.sleep(1)
    adc = 0 # pin GP26
    await board.set_pin_mode_analog_input(adc, differential=0, callback=the_callback)

loop.run_until_complete(addSensor(board))



Run for some time

In [16]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))

a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 66  v: 0.053173828124999996 r: 0.01611328125 fac: 61.06060606060606, ldr: 610606.0606060607
a: 67  v: 0.0539794921875 r: 0.016357421875 fac: 60.134328358208954, ldr: 601343.2835820896
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.8461538462
a: 65  v: 0.0523681640625 r: 0.015869140625 fac: 62.01538461538462, ldr: 620153.84615